### Setup the connection 

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Myrto.Setzi/Ironhack Data Analytics.json"

In [4]:
from google.cloud import bigquery

In [5]:
client = bigquery.Client()

### Challenge 1 - Most Profiting Authors?

### Step 1

In [10]:
query_1='''
SELECT 
    t.title_id
    , t.price
    , t.advance
    , t.royalty
    , s.qty
    , a.au_id
    , au_lname
    , au_fname
    , ta.royaltyper
    , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
FROM  
    `ironhack-data-analytics.publications.titles` t
INNER JOIN 
    `ironhack-data-analytics.publications.sales` s
ON
    s.title_id = t.title_id
INNER JOIN 
    `ironhack-data-analytics.publications.titleauthor` ta
ON 
   ta.title_id = s.title_id
INNER JOIN
    `ironhack-data-analytics.publications.authors` a
ON 
    a.au_id = ta.au_id
ORDER BY
    t.title_id
    , a.au_id
'''

In [11]:
query_job_1 = client.query(query=query_1)

In [12]:
df_1=query_job_1.to_dataframe()

In [13]:
df_1.head(10)

,title_id,price,advance,royalty,qty,au_id,au_lname,au_fname,royaltyper,royalties
0,BU1032,19.99,5000.0,10,5,213-46-8915,Green,Marjorie,40,3.998
1,BU1032,19.99,5000.0,10,10,213-46-8915,Green,Marjorie,40,7.996
2,BU1032,19.99,5000.0,10,5,409-56-7008,Bennet,Abraham,60,5.997
3,BU1032,19.99,5000.0,10,10,409-56-7008,Bennet,Abraham,60,11.994
4,BU1111,11.95,5000.0,10,25,267-41-2394,O'Leary,Michael,40,11.950
5,BU1111,11.95,5000.0,10,25,724-80-9391,MacFeather,Stearns,60,17.925
6,BU2075,2.99,10125.0,24,35,213-46-8915,Green,Marjorie,100,25.116
7,BU7832,19.99,5000.0,10,15,274-80-9391,Straight,Dean,100,29.985
8,MC2222,19.99,0.0,12,10,712-45-1867,del Castillo,Innes,100,23.988
9,MC3021,2.99,15000.0,24,25,722-51-5454,DeFrance,Michel,75,13.455


### Step 2

In [14]:
query_2='''
SELECT
    title_id
    , au_id
    , au_lname
    , au_fname
    , advance
    , sum(royalties) AS royalties
FROM
    (SELECT 
        t.title_id
        , t.price
        , t.advance
        , t.royalty
        , s.qty
        , a.au_id
        , au_lname
        , au_fname
        , ta.royaltyper
        , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
    FROM  
        `ironhack-data-analytics.publications.titles` t
    INNER JOIN 
        `ironhack-data-analytics.publications.sales` s
    ON
        s.title_id = t.title_id
    INNER JOIN 
        `ironhack-data-analytics.publications.titleauthor` ta
    ON 
       ta.title_id = s.title_id
    INNER JOIN
        `ironhack-data-analytics.publications.authors` a
    ON 
        a.au_id = ta.au_id
    ORDER BY
        t.title_id
        , a.au_id) roy
GROUP BY
    1,2,3,4,5
'''



In [15]:
query_job_2 = client.query(query=query_2)

In [18]:
df_2=query_job_2.to_dataframe()

In [19]:
df_2.head(10)

,title_id,au_id,au_lname,au_fname,advance,royalties
0,BU1032,213-46-8915,Green,Marjorie,5000.0,11.994
1,MC3021,899-46-2035,Ringer,Anne,15000.0,7.176
2,MC3021,722-51-5454,DeFrance,Michel,15000.0,21.528
3,PC8888,846-92-7186,Hunter,Sheryl,8000.0,50.000
4,PS1372,724-80-9391,MacFeather,Stearns,7000.0,10.795
5,TC3218,807-91-6654,Panteley,Sylvia,7000.0,83.800
6,BU7832,274-80-9391,Straight,Dean,5000.0,29.985
7,PC8888,427-17-2319,Dull,Ann,8000.0,50.000
8,BU1032,409-56-7008,Bennet,Abraham,5000.0,17.991
9,TC7777,472-27-2349,Gringlesby,Burt,8000.0,8.994


### Step 3

In [20]:
query_3='''
SELECT
    au_id AS author_id
    , au_lname AS last_name
    , au_fname AS first_name
    , sum(advance + royalties) AS profits 
from 
    (SELECT
        title_id
        , au_id
        , au_lname
        , au_fname
        , advance
        , sum(royalties) AS royalties
    FROM
        (SELECT 
            t.title_id
            , t.price
            , t.advance
            , t.royalty
            , s.qty
            , a.au_id
            , au_lname
            , au_fname
            , ta.royaltyper
            , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
        FROM  
            `ironhack-data-analytics.publications.titles` t
        INNER JOIN 
            `ironhack-data-analytics.publications.sales` s
        ON
            s.title_id = t.title_id
        INNER JOIN 
            `ironhack-data-analytics.publications.titleauthor` ta
        ON 
           ta.title_id = s.title_id
        INNER JOIN
            `ironhack-data-analytics.publications.authors` a
        ON 
            a.au_id = ta.au_id
        ORDER BY
            t.title_id
            , a.au_id) roy
    GROUP BY
        1,2,3,4,5)prof
GROUP BY
    1,2,3
ORDER BY
    profits DESC
LIMIT 3
'''



In [21]:
query_job_3 = client.query(query=query_3)

In [22]:
df_3=query_job_3.to_dataframe()

In [23]:
df_3.head(10)

,author_id,last_name,first_name,profits
0,899-46-2035,Ringer,Anne,17353.132
1,213-46-8915,Green,Marjorie,15162.110
2,722-51-5454,DeFrance,Michel,15021.528


### Challenge 2 - Most Profiting Authors Table

In [ ]:
query_4='''
CREATE TABLE publications.most_profiting_authors AS
    au_id 
    , sum(advance + royalties) AS profits 
from 
    (SELECT
        title_id
        , au_id
        , au_lname
        , au_fname
        , advance
        , sum(royalties) AS royalties
    FROM
        (SELECT 
            t.title_id
            , t.price
            , t.advance
            , t.royalty
            , s.qty
            , a.au_id
            , au_lname
            , au_fname
            , ta.royaltyper
            , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
        FROM  
            `ironhack-data-analytics.publications.titles` t
        INNER JOIN 
            `ironhack-data-analytics.publications.sales` s
        ON
            s.title_id = t.title_id
        INNER JOIN 
            `ironhack-data-analytics.publications.titleauthor` ta
        ON 
           ta.title_id = s.title_id
        INNER JOIN
            `ironhack-data-analytics.publications.authors` a
        ON 
            a.au_id = ta.au_id
        ORDER BY
            t.title_id
            , a.au_id) roy
    GROUP BY
        1,2,3,4,5)prof
GROUP BY
    1
'''


In [ ]:
query_job_4 = client.query(query=query_4)